In [1]:
import sys
sys.path.insert(1, "/home/odyssey/mmk_smoke_detection/")

In [2]:
import os 
from tensorflow import keras
import tensorflow as tf
import shutil
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2021-11-11 01:20:58.396261: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-11 01:20:58.480588: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-11 01:20:58.480740: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
!nvidia-smi

Thu Nov 11 01:20:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| N/A   59C    P8    12W /  N/A |    161MiB /  7973MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
PATH_TO_BINARY_MEDIUM_DATASET = "/home/odyssey/mmk_smoke_detection/augmentation/binary_splitted_medium_aug_dataset/"
PATH_TO_MEDIUM_DATASET = "/home/odyssey/mmk_smoke_detection/augmentation/splitted_medium_aug_dataset/"

PATH_TO_MERGED_BINARY_SMALL_DATASET = "/home/odyssey/mmk_smoke_detection/augmentation/binary_splitted_small_aug_dataset_2_2/"
PATH_TO_MERGED_SMALL_DATASET = "/home/odyssey/mmk_smoke_detection/augmentation/merged_small_aug_dataset_2/"
PATH_TO_SMALL_DATASET = "/home/odyssey/mmk_smoke_detection/augmentation/splitted_small_aug_dataset/"
PATH_TO_BINARY_SMALL_DATASET = "/home/odyssey/mmk_smoke_detection/augmentation/binary_splitted_small_aug_dataset/"

PATH_TO_THREE_SMALL_DATASET = "/home/odyssey/mmk_smoke_detection/augmentation/three_small_aug_dataset/"

PATH_TO_VALIDATION_DATASET = "/home/odyssey/mmk_smoke_detection/validation/expanded_val/"
PATH_TO_THREE_VAL_DATASET = "/home/odyssey/mmk_smoke_detection/validation/line_three_val/"
PATH_TO_BINARY_VAL_DATASET = "/home/odyssey/mmk_smoke_detection/validation/line_binary_val/"

PATH_TO_HANDLED_THREE_VAL_DATASET = "/home/odyssey/mmk_smoke_detection/dataset_preparator/"
PATH_TO_HANDLED_BINARY_VAL_DATASET = "/home/odyssey/mmk_smoke_detection/quality/binary_handled_validation/"

# LABELS = ['background', 'emission', 'fire', 'machine']
LABELS = ['background', 'emission', 'fire']
N, M = 224, 224

LOG_DIR = "./mlogs"

In [9]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.figsize'] = (18, 8)
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False 

def plot_lr(initial_history, epoch_num):
    plt.plot(
        np.arange(1, epoch_num + 1), 
        initial_history.history['val_loss'], 
        label='Loss', lw=3
    )
    plt.plot(
        np.arange(1, epoch_num + 1), 
        initial_history.history['val_binary_accuracy'], 
        label='Accuracy', lw=3
    )
    plt.plot(
        np.arange(1, epoch_num + 1), 
        initial_history.history['lr'], 
        label='Learning rate', color='#000', lw=3, linestyle='--'
    )
    plt.title('Evaluation metrics', size=20)
    plt.xlabel('Epoch', size=14)
    plt.legend()

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# def flow_from_directory(
#     path_to_dataset: str,
#     is_binary: bool
# ):
#     keras_ds = tf.keras.preprocessing.image_dataset_from_directory(
#         directory=path_to_dataset,
#         labels='inferred',
#         label_mode='binary' if is_binary else 'categorical',
#         class_names=LABELS[:2] if is_binary else LABELS, 
#         batch_size=32, 
#         image_size=(N, M)
#     )
#     keras_ds = keras_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
#     return keras_ds


def flow_from_directory(
    path_to_dataset: str,
    is_binary: bool,
    is_valid: bool = False,
    is_fire: bool = False
) -> ImageDataGenerator:
    img_gen = ImageDataGenerator(
        featurewise_std_normalization=True,
        featurewise_center=True,
        rotation_range=15,
        width_shift_range=10,
        height_shift_range=10,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
    ) if not is_valid else ImageDataGenerator()
    if is_binary:
        classes = [LABELS[0], LABELS[2]] if is_fire else LABELS[:2]
    else:
        classes = LABELS
    keras_ds = img_gen.flow_from_directory(
        directory=path_to_dataset,
        target_size=(N, M),
        classes=classes,
        class_mode='binary' if is_binary else 'categorical',
        batch_size=32,
    ) 
    return keras_ds

In [23]:
from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import math

from typing import Optional, Callable


def gen_metrics(
    is_binary: bool
) -> list:
    if is_binary:
        return [
            keras.metrics.BinaryAccuracy(),
            keras.metrics.AUC(),
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall')
        ]
    
    cbs = [keras.metrics.CategoricalAccuracy(),]
    for idx, label in enumerate(LABELS):
        cbs.append(keras.metrics.Precision(
            class_id=idx,
            name=f'precision_{label}'
        ))
        cbs.append(keras.metrics.Recall(
            class_id=idx,
            name=f'recall_{label}'
        ))
    return cbs

def freeze_layers(
    base_model: Model,
    freeze_degree: float
):
    layers_num = len(base_model.layers)
    layers_border = int(freeze_degree * layers_num)
    print("Layers border:", layers_border)
    for idx in range(layers_border):
        base_model.layers[idx].trainable = False

def prepare_model(
    base_model: Model,
    freeze_degree: float,
    is_binary: bool = False,
) -> Model:
    freeze_layers(base_model, freeze_degree)
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
#     x = tf.keras.layers.Dense(128, activation='relu')(x)
#     x = tf.keras.layers.Dropout(rate=0.7)(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
#     x = tf.keras.layers.Dropout(rate=0.5)(x)
    if is_binary:
        # TODO попробовать сменить на log
        predictions = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    else:
        predictions = tf.keras.layers.Dense(len(LABELS), activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    opt = keras.optimizers.Adam(learning_rate=1e-04)
#     opt = keras.optimizers.SGD(learning_rate=1e-02)
    model.compile(
        optimizer=opt, 
        loss='binary_crossentropy' if is_binary else 'categorical_crossentropy',
        metrics=gen_metrics(is_binary)
    )
    return model

def choose_dataset(is_binary: bool, is_medium: bool) -> str:
    # PATH_TO_BINARY_MEDIUM_DATASET = "/home/odyssey/mmk_smoke_detection/augmentation/binary_splitted_medium_aug_dataset/"
    # PATH_TO_MEDIUM_DATASET = "/home/odyssey/mmk_smoke_detection/augmentation/splitted_medium_aug_dataset/"
    # PATH_TO_SMALL_DATASET = "/home/odyssey/mmk_smoke_detection/augmentation/splitted_small_aug_dataset/"
    # PATH_TO_BINARY_SMALL_DATASET = "/home/odyssey/mmk_smoke_detection/augmentation/binary_splitted_small_aug_dataset/"
    if is_binary:
#         return PATH_TO_BINARY_MEDIUM_DATASET if is_medium else PATH_TO_BINARY_SMALL_DATASET
        return PATH_TO_BINARY_MEDIUM_DATASET if is_medium else PATH_TO_MERGED_BINARY_SMALL_DATASET
    else:
        return PATH_TO_MEDIUM_DATASET  if is_medium else PATH_TO_MERGED_SMALL_DATASET
    
def choose_val_dataset(is_binary: bool) -> str:
#     return PATH_TO_BINARY_VAL_DATASET if is_binary else PATH_TO_THREE_VAL_DATASET
    return PATH_TO_HANDLED_BINARY_VAL_DATASET if is_binary else PATH_TO_HANDLED_THREE_VAL_DATASET

def default_step_decay(epoch):
#     initial_lrate = 1e-3
#     drop = 0.5
#     epochs_drop = 4
#     lrate = initial_lrate * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
    initial_lrate = 1e-3
    drop = 0.5
    epochs_drop = 4
    lrate = initial_lrate * math.pow(drop, math.floor((epoch + 1) / epochs_drop))
    return lrate
    
def fit_model(
    base_model: Model,
    log_path: str,
    epochs_num: int, 
    is_medium: bool = False,
    is_binary: bool = False,
    is_fire: bool = False,
    path_to_dataset: Optional[str] = None,
    path_to_val_dataset: Optional[str] = None,
    freeze_degree: float = 1.0,
    path_to_checkpoint: Optional[str] = None,
    step_decay: Callable = default_step_decay
):
    if path_to_checkpoint:
        if os.path.exists(path_to_checkpoint):
            shutil.rmtree(path_to_checkpoint)
        os.makedirs(path_to_checkpoint)
        
    path_to_dataset = path_to_dataset or choose_dataset(is_binary, is_medium)
    path_to_val_dataset = path_to_val_dataset or choose_val_dataset(is_binary)
    train_set = flow_from_directory(path_to_dataset=os.path.join(path_to_dataset, 'train'), is_binary=is_binary, is_fire=is_fire)
#     test_set = flow_from_directory(path_to_dataset=os.path.join(path_to_dataset, 'test'), is_binary=is_binary)
#     test_set = flow_from_directory(path_to_dataset=os.path.join(path_to_dataset, 'test'), is_binary=is_binary)
    val_set = flow_from_directory(
        path_to_dataset=path_to_val_dataset, 
        is_binary=is_binary,
        is_valid=True,
        is_fire=is_fire
    )
    
    callbacks = [
        keras.callbacks.TensorBoard(log_path),
        keras.callbacks.LearningRateScheduler(step_decay),
#         keras.callbacks.EarlyStopping(
#             monitor="val_loss",
#             min_delta=0.002,
#             patience=5,
#         )
    ]
    if path_to_checkpoint:
        if is_binary:
            chekpoint_file = "{epoch:02d}-{val_precision:.2f}-{val_recall:.2f}.hdf5"
        else:
            chekpoint_file = "{epoch:02d}-{val_precision_background:.2f}-{val_recall_background:.2f}-{val_precision_precision:.2f}-{val_recall_precision:.2f}-{val_precision_fire:.2f}-{val_recall_fire:.2f}.hdf5"
        callbacks.append(
            keras.callbacks.ModelCheckpoint(filepath=os.path.join(path_to_checkpoint, chekpoint_file))
        )
    
    model = prepare_model(base_model=base_model, is_binary=is_binary, freeze_degree=freeze_degree)
    history = model.fit(
        train_set,
        epochs=epochs_num,
        callbacks=callbacks,
        validation_data=val_set
    )
    return history

    

In [27]:
model_name = "efficient_net_B0"
base_model = tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
train_path = "/home/odyssey/mmk_smoke_detection/augmentation/line_binary_small_aug_dataset_extended/"
val_path = "/home/odyssey/mmk_smoke_detection/validation/line_binary_val/"
path_to_checkpoint = f"/home/odyssey/mmk_smoke_detection/quality/checkpoints/{model_name}_small/"

epoch_num = 30
val_metrics = fit_model(
    base_model=base_model,
    log_path=f'./{model_name}_imagenet_line_binary_clean_08_small_logs',
    epochs_num=epoch_num,
    freeze_degree=0.8,
    path_to_dataset=train_path,
    path_to_val_dataset=val_path,
    is_binary=True,
    path_to_checkpoint=path_to_checkpoint
)

Found 20659 images belonging to 2 classes.
Found 638 images belonging to 2 classes.
Layers border: 189
Epoch 1/30


KeyboardInterrupt: 

In [18]:
# # MY_PATH_TO_DATASET = "/home/odyssey/mmk_smoke_detection/augmentation/three_merged_ready_dataset_copy"
# # MY_PATH_TO_DATASET = "/home/odyssey/mmk_smoke_detection/augmentation/line_binary_small_aug_dataset/"
# # MY_PATH_TO_DATASET = "/home/odyssey/mmk_smoke_detection/augmentation/line_binary_extra_small_aug_dataset/" # ТОП с rt аугменьацией
# # MY_PATH_TO_DATASET = "/home/odyssey/mmk_smoke_detection/augmentation/line_binary_small_aug_dataset_extended/" # ТОП с rt аугменьацией
# MY_PATH_TO_DATASET = "/home/odyssey/mmk_smoke_detection/augmentation/line_binary_large_aug_dataset_extended/" # ТОП с rt аугменьацией
# # MY_PATH_TO_DATASET = "/home/odyssey/mmk_smoke_detection/augmentation/line_binary_extra_small_aug_dataset_2/"
# LINE_VAL_THREE_DATASET = "/home/odyssey/mmk_smoke_detection/validation/line_binary_val/"
        

# RESULT_FILE = 'result.txt'
# models = [
# #     ('resnet50', tf.keras.applications.resnet.ResNet50(weights=None, include_top=False, input_shape=(224, 224, 3))),
# #     ('resnet50', tf.keras.applications.resnet.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
# #     ('resnetv2_101_2', tf.keras.applications.resnet_v2.ResNet101V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
# #     TOP 16 e('efficient_net_B6', tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
#     ('efficient_net_B0', tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
# #     ('efficient_net_B2', tf.keras.applications.EfficientNetB2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
# #     ('inception_resnet_v2', tf.keras.applications.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
# #     ('inception_v3', tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
# #     ('xception', tf.keras.applications.Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
# #     ('densenet', tf.keras.applications.DenseNet121(weights=None, include_top=False, input_shape=(224, 224, 3))),
# #     ('vgg', tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
# #     ('mobilenet', tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
# #     ('efficient_net_v2_S', tf.keras.applications.efficientnet_v2.EfficientNetV2S(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
# ]

model_name = "efficient_net_B0"
base_model = tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
train_path = "/home/odyssey/mmk_smoke_detection/augmentation/line_binary_large_aug_dataset_extended/"
val_path = "/home/odyssey/mmk_smoke_detection/validation/line_binary_val/"
path_to_checkpoint = f"/home/odyssey/mmk_smoke_detection/quality/checkpoints/{model_name}_large/"

epoch_num = 30
val_metrics = fit_model(
    base_model=base_model,
    log_path=f'./{model_name}_imagenet_line_binary_clean_08_large_logs',
    epochs_num=epoch_num,
    freeze_degree=0.8,
    path_to_dataset=train_path,
    path_to_val_dataset=val_path,
    is_binary=True,
    path_to_checkpoint=path_to_checkpoint
)

Found 82636 images belonging to 2 classes.
Found 638 images belonging to 2 classes.
Layers border: 189
Epoch 1/30
2583/2583 [==============================] - 535s 205ms/step - loss: 0.1993 - binary_accuracy: 0.9245 - auc_5: 0.9759 - precision: 0.9160 - recall: 0.9374 - val_loss: 0.3186 - val_binary_accuracy: 0.8966 - val_auc_5: 0.9459 - val_precision: 0.8303 - val_recall: 0.7829 - lr: 0.0010
Epoch 2/30
2583/2583 [==============================] - 525s 203ms/step - loss: 0.1396 - binary_accuracy: 0.9482 - auc_5: 0.9875 - precision: 0.9411 - recall: 0.9580 - val_loss: 0.3493 - val_binary_accuracy: 0.9138 - val_auc_5: 0.9512 - val_precision: 0.9545 - val_recall: 0.7200 - lr: 0.0010
Epoch 3/30
2583/2583 [==============================] - 520s 201ms/step - loss: 0.1229 - binary_accuracy: 0.9537 - auc_5: 0.9901 - precision: 0.9475 - recall: 0.9621 - val_loss: 0.3030 - val_binary_accuracy: 0.9232 - val_auc_5: 0.9447 - val_precision: 0.9437 - val_recall: 0.7657 - lr: 0.0010- loss: 0.1232 - bi

KeyboardInterrupt: 

In [19]:
model_name = "efficient_net_B0"
base_model = tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
train_path = "/home/odyssey/mmk_smoke_detection/augmentation/line_binary_extra_large_aug_dataset_extended/"
val_path = "/home/odyssey/mmk_smoke_detection/validation/line_binary_val/"
path_to_checkpoint = f"/home/odyssey/mmk_smoke_detection/quality/checkpoints/{model_name}_extra_large/"

epoch_num = 17
val_metrics = fit_model(
    base_model=base_model,
    log_path=f'./{model_name}_imagenet_line_binary_clean_08_small_logs',
    epochs_num=epoch_num,
    freeze_degree=0.8,
    path_to_dataset=train_path,
    path_to_val_dataset=val_path,
    is_binary=True,
    path_to_checkpoint=path_to_checkpoint
)

Found 165272 images belonging to 2 classes.
Found 638 images belonging to 2 classes.
Layers border: 189
Epoch 1/17
5165/5165 [==============================] - 1013s 195ms/step - loss: 0.1755 - binary_accuracy: 0.9343 - auc_6: 0.9809 - precision: 0.9275 - recall: 0.9444 - val_loss: 0.3776 - val_binary_accuracy: 0.9122 - val_auc_6: 0.9193 - val_precision: 0.8742 - val_recall: 0.7943 - lr: 0.0010
Epoch 2/17
5165/5165 [==============================] - 1005s 195ms/step - loss: 0.1208 - binary_accuracy: 0.9547 - auc_6: 0.9903 - precision: 0.9487 - recall: 0.9628 - val_loss: 0.3469 - val_binary_accuracy: 0.9154 - val_auc_6: 0.9424 - val_precision: 0.8712 - val_recall: 0.8114 - lr: 0.0010
Epoch 3/17
5165/5165 [==============================] - 1002s 194ms/step - loss: 0.1043 - binary_accuracy: 0.9600 - auc_6: 0.9926 - precision: 0.9535 - recall: 0.9685 - val_loss: 0.3416 - val_binary_accuracy: 0.9201 - val_auc_6: 0.9495 - val_precision: 0.9079 - val_recall: 0.7886 - lr: 0.0010
Epoch 4/17
516

In [17]:
model_name = "inception_v3"
base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
train_path = "/home/odyssey/mmk_smoke_detection/augmentation/line_binary_small_aug_dataset_extended/"
val_path = "/home/odyssey/mmk_smoke_detection/validation/line_binary_val/"
path_to_checkpoint = f"/home/odyssey/mmk_smoke_detection/quality/checkpoints/{model_name}_small/"

epoch_num = 30
val_metrics = fit_model(
    base_model=base_model,
    log_path=f'./{model_name}_imagenet_line_binary_clean_08_small_logs',
    epochs_num=epoch_num,
    freeze_degree=0.8,
    path_to_dataset=train_path,
    path_to_val_dataset=val_path,
    is_binary=True,
    path_to_checkpoint=path_to_checkpoint
)

Found 20659 images belonging to 2 classes.
Found 638 images belonging to 2 classes.
Layers border: 248
Epoch 1/30
646/646 [==============================] - 141s 210ms/step - loss: 0.5344 - binary_accuracy: 0.7430 - auc_4: 0.8151 - precision: 0.7387 - recall: 0.7646 - val_loss: 0.5859 - val_binary_accuracy: 0.7163 - val_auc_4: 0.7700 - val_precision: 0.4766 - val_recall: 0.3486 - lr: 0.0010
Epoch 2/30
646/646 [==============================] - 135s 209ms/step - loss: 0.4372 - binary_accuracy: 0.8126 - auc_4: 0.8858 - precision: 0.7945 - recall: 0.8514 - val_loss: 0.4843 - val_binary_accuracy: 0.7727 - val_auc_4: 0.8337 - val_precision: 0.6974 - val_recall: 0.3029 - lr: 0.0010
Epoch 3/30
646/646 [==============================] - 135s 209ms/step - loss: 0.4004 - binary_accuracy: 0.8367 - auc_4: 0.9058 - precision: 0.8256 - recall: 0.8602 - val_loss: 0.5051 - val_binary_accuracy: 0.7571 - val_auc_4: 0.8209 - val_precision: 0.5926 - val_recall: 0.3657 - lr: 0.00109s - lo
Epoch 4/30
116/64

KeyboardInterrupt: 

In [16]:
model_name = "efficient_net_B0"
base_model = tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
train_path = "/home/odyssey/mmk_smoke_detection/augmentation/line_three_large_aug_dataset/"
val_path = "/home/odyssey/mmk_smoke_detection/validation/line_three_val/"
path_to_checkpoint = f"/home/odyssey/mmk_smoke_detection/quality/checkpoints/{model_name}_three_large/"

def step_decay(epoch):
#     initial_lrate = 1e-3
#     drop = 0.5
#     epochs_drop = 4
#     lrate = initial_lrate * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
    initial_lrate = 1e-3
    drop = 0.5
    epochs_drop = 4
    lrate = initial_lrate * math.pow(drop, math.floor((epoch + 1) / epochs_drop))
    return lrate

epoch_num = 30
val_metrics = fit_model(
    base_model=base_model,
    log_path=f'./{model_name}_imagenet_line_three_08_large_logs',
    epochs_num=epoch_num,
    freeze_degree=0.8,
    path_to_dataset=train_path,
    path_to_val_dataset=val_path,
#     path_to_checkpoint=path_to_checkpoint,    
)

Found 72256 images belonging to 3 classes.
Found 638 images belonging to 3 classes.
Layers border: 189
Epoch 1/30
2258/2258 [==============================] - 487s 214ms/step - loss: 0.3159 - categorical_accuracy: 0.8891 - precision_background: 0.8711 - recall_background: 0.8096 - precision_emission: 0.8927 - recall_emission: 0.8868 - precision_fire: 0.9406 - recall_fire: 0.9242 - val_loss: 1.5448 - val_categorical_accuracy: 0.6129 - val_precision_background: 0.7381 - val_recall_background: 0.3185 - val_precision_emission: 0.8544 - val_recall_emission: 0.7714 - val_precision_fire: 0.4620 - val_recall_fire: 0.9240 - lr: 0.0010
Epoch 2/30
2258/2258 [==============================] - 475s 210ms/step - loss: 0.2247 - categorical_accuracy: 0.9256 - precision_background: 0.9096 - recall_background: 0.8754 - precision_emission: 0.9252 - recall_emission: 0.9286 - precision_fire: 0.9590 - recall_fire: 0.9541 - val_loss: 1.2556 - val_categorical_accuracy: 0.6536 - val_precision_background: 0.837

KeyboardInterrupt: 

In [25]:
# models = [
# #     ('resnet50', tf.keras.applications.resnet.ResNet50(weights=None, include_top=False, input_shape=(224, 224, 3))),
# #     ('resnet50', tf.keras.applications.resnet.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
# #     ('resnetv2_101_2', tf.keras.applications.resnet_v2.ResNet101V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
# #     TOP 16 e('efficient_net_B6', tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
#     ('efficient_net_B0', tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
# #     ('efficient_net_B2', tf.keras.applications.EfficientNetB2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
# #     ('inception_resnet_v2', tf.keras.applications.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
# #     ('inception_v3', tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
# #     ('xception', tf.keras.applications.Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
# #     ('densenet', tf.keras.applications.DenseNet121(weights=None, include_top=False, input_shape=(224, 224, 3))),
# #     ('vgg', tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
# #     ('mobilenet', tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
# #     ('efficient_net_v2_S', tf.keras.applications.efficientnet_v2.EfficientNetV2S(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
# ]
#     ('xception', tf.keras.applications.Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),

# попытаюсь разделить background и fire
! ls /home/odyssey/mmk_smoke_detection/augmentation/line_fire_binary_dataset/train
# model_name = "xception"
# base_model = tf.keras.applications.Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# train_path = "/home/odyssey/mmk_smoke_detection/augmentation/line_fire_binary_dataset/"
model_name = "densenet"
base_model = tf.keras.applications.DenseNet121(weights=None, include_top=False, input_shape=(224, 224, 3))
train_path = "/home/odyssey/mmk_smoke_detection/augmentation/line_small_fire_binary_dataset/"
val_path = "/home/odyssey/mmk_smoke_detection/validation/line_fire_binary_val/"
path_to_checkpoint = f"/home/odyssey/mmk_smoke_detection/quality/checkpoints/{model_name}_three_large/"

def step_decay(epoch):
#     initial_lrate = 1e-3
#     drop = 0.5
#     epochs_drop = 4
#     lrate = initial_lrate * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
    initial_lrate = 1e-2
    drop = 0.5
    epochs_drop = 5
    lrate = initial_lrate * math.pow(drop, math.floor((epoch + 1) / epochs_drop))
    return lrate

epoch_num = 10
val_metrics = fit_model(
    base_model=base_model,
    log_path=f'./{model_name}_imagenet_fire_05_logs',
    epochs_num=epoch_num,
    freeze_degree=0.0,
    path_to_dataset=train_path,
    path_to_val_dataset=val_path,
    is_binary=True,
    is_fire=True
#     path_to_checkpoint=path_to_checkpoint,    
)

background  fire
Found 49802 images belonging to 2 classes.
Found 463 images belonging to 2 classes.
Layers border: 0
Epoch 1/10
1557/1557 [==============================] - 458s 287ms/step - loss: 0.2112 - binary_accuracy: 0.9297 - auc_8: 0.9690 - precision: 0.9387 - recall: 0.9274 - val_loss: 1.4707 - val_binary_accuracy: 0.5443 - val_auc_8: 0.7050 - val_precision: 0.4419 - val_recall: 0.8889 - lr: 0.0010
Epoch 2/10
1557/1557 [==============================] - 447s 287ms/step - loss: 0.1623 - binary_accuracy: 0.9481 - auc_8: 0.9800 - precision: 0.9561 - recall: 0.9450 - val_loss: 1.3729 - val_binary_accuracy: 0.5896 - val_auc_8: 0.7080 - val_precision: 0.4700 - val_recall: 0.8713 - lr: 0.0010
Epoch 3/10
1557/1557 [==============================] - 447s 287ms/step - loss: 0.1480 - binary_accuracy: 0.9523 - auc_8: 0.9833 - precision: 0.9613 - recall: 0.9478 - val_loss: 1.7508 - val_binary_accuracy: 0.5184 - val_auc_8: 0.7408 - val_precision: 0.4333 - val_recall: 0.9883 - lr: 0.0010
Epo

KeyboardInterrupt: 